# The Random behavior of assets 

# The Random behavior of assets
<p float = "Left">
    <img src="stock_market_prices.jpg" style="width:1400px;height:400px">
</p>

<p>The image above on the left, represents a  Browian Motion with pollen particles discovered by Robert Brown. This was the start to a new theory about random variables and stochastic calculus

<p>In this first projet, we are going to verify the limitations of considering stock prices daily returns Normaly distributed.</p>

<p>After that, we are going to have a look at Stocastic Calculus and itos Lema, to simulate the behavior of stock price in the time</p>


## 1. Importing all the needed information
<p> We are going to use the following libraries for this project:
<ul>
    <li>Pandas: This package is used to manipulate the data</li>
    <li>Numpy: This package is used to calculate basic results</li>
    <li>Seaborn: This package is used to plot the data</li>
    <li>Yfinance: This package is used to dowload the financial data for a given asset </li>
</ul>
To convert our dictionary  DataFrame, we will first need to import the library under its usual alias. We'll also want to inspect our DataFrame to ensure it was created correctly. Let's perform these steps now.</p>

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# import seaborn as sns 
import yfinance as yf
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import cufflinks as cf
import plotly.express as px
cf.set_config_file(offline=True)


In [69]:
stock_name = 'AMZN'
df_stk = yf.download(stock_name, start="1980-01-01", end="2020-04-30")

[*********************100%***********************]  1 of 1 completed


In [70]:
df_stk['Adj Close'].iplot(kind='line')

In [71]:
df_stk['Date'] = df_stk.index
df_stk_rel_cols = df_stk[['Date','Adj Close']]
df_stk_rel_cols['Adj Close Day Before'] = df_stk_rel_cols['Adj Close'].shift(1) 
df_stk_rel_cols['Daily Returns Perc'] = ((df_stk_rel_cols['Adj Close'] - df_stk_rel_cols['Adj Close Day Before'])/df_stk_rel_cols['Adj Close Day Before'])
df_stk_rel_cols['Daily Returns'] = 1 + df_stk_rel_cols['Daily Returns Perc'] 
df_stk_rel_cols['Log Daily Returns'] = np.log(df_stk_rel_cols['Daily Returns'])
df_stk_rel_cols['Cumulative Log Daily Returns'] = df_stk_rel_cols['Log Daily Returns'].cumsum()
df_stk_rel_cols['ExP Cumulative Log Daily Returns'] = np.exp(df_stk_rel_cols['Cumulative Log Daily Returns'])

In [72]:
df_stk_rel_cols['Daily Returns'].iplot(kind='scatter',size=1,mode='markers',color = 'Blue')
plt.show()

In [73]:
df_stk_rel_cols.head()

,Date,Adj Close,Adj Close Day Before,Daily Returns Perc,Daily Returns,Log Daily Returns,Cumulative Log Daily Returns,ExP Cumulative Log Daily Returns
Date,,,,,,,,
1997-05-15,1997-05-15,1.958333,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-16,1997-05-16,1.729167,1.958333,-0.117021,0.882979,-0.124454,-0.124454,0.882979
1997-05-19,1997-05-19,1.708333,1.729167,-0.012049,0.987951,-0.012122,-0.136576,0.872340
1997-05-20,1997-05-20,1.635417,1.708333,-0.042683,0.957317,-0.043620,-0.180196,0.835107
1997-05-21,1997-05-21,1.427083,1.635417,-0.127389,0.872611,-0.136265,-0.316461,0.728723


### Calculating Mean, Std, Skewness and Kurtosis

In [74]:
daily_returns = df_stk_rel_cols['Daily Returns Perc'].dropna()
mean_daily_return = np.mean(daily_returns)
var_daily_return =  np.sum((daily_returns - mean_daily_return)**2 )/(len(daily_returns) - 1)
std_daily_return = np.sqrt(var_daily_return)

skew_daily_return = np.sum((daily_returns - mean_daily_return)**3 )/( (std_daily_return**3)*(len(daily_returns) - 1))
kurt_daily_return = np.sum((daily_returns - mean_daily_return)**4 )/( (std_daily_return**4)*(len(daily_returns) - 1))
years_of_data =  np.round((max(df_stk_rel_cols['Date']) - min(df_stk_rel_cols['Date']) ).days /365 ,0)
print(f'Number of years in the analysis: {years_of_data}')
print(f'Average daily return: {round(mean_daily_return*100,4)}%')
print(f'Standard Deviation daily return: {round(std_daily_return*100,4)}%')
print(f'Skewness daily return: {round(skew_daily_return,2)}')
print(f'Kurtosis daily return: {round(kurt_daily_return,3)}')


Number of years in the analysis: 23.0
Average daily return: 0.1913%
Standard Deviation daily return: 3.7341%
Skewness daily return: 1.05
Kurtosis daily return: 13.408


In [75]:
fig = px.histogram(df_stk_rel_cols, x="Daily Returns Perc")
fig.show()

In [77]:
# Create figure with secondary y-axis
df_stk_rel_cols['Normalized Daily Returns'] = (df_stk_rel_cols['Daily Returns Perc'] - mean_daily_return)/std_daily_return 
distributions = df_stk_rel_cols['Normalized Daily Returns'].dropna()
x_norm = np.random.normal(loc=0,scale=1,size=10000)#* + 
fig = ff\
        .create_distplot(
            [x_norm,distributions],
            [' Equivalent Normal Distribution','Daily Returns PDF'],
            # show_rug = True,
            histnorm  = 'probability',
            show_hist = False,
            show_curve = True
            
        )\
        .update_layout(
            title = 'Daily Returns PDF vs Equivalent Normal Distribution',
            xaxis_title ='Daily Returns',
            yaxis_title = 'Probability'
        )\
        .update_xaxes(range=[-5, 5 ])

fig.show()

In [78]:
extreme_results_99 = [ 1 if (d_r <= mean_daily_return - 2.56*std_daily_return or d_r >= mean_daily_return + 2.56*std_daily_return ) else 0 for d_r in  daily_returns ]
extreme_results_9999 = [ 1 if (d_r <= mean_daily_return - 3*std_daily_return or d_r >= mean_daily_return + 3*std_daily_return ) else 0 for d_r in  daily_returns ]
extreme_results_999999 = [ 1 if (d_r <= mean_daily_return - 7*std_daily_return or d_r >= mean_daily_return + 7*std_daily_return ) else 0 for d_r in  daily_returns ]

print(f'Number of values outside the 99% confidence interval (It should happen 3 times in a year ) : {np.sum(extreme_results_99)}')
print(f'Number of values outside the 99.99% confidence interval (It should happen once in 4 years) : {np.sum(extreme_results_9999)}')
print(f'Number of values outside the 99.999999999999% confidence interval (It should happen twice in a million years): {np.sum(extreme_results_999999)}')

Number of values outside the 99% confidence interval (It should happen 3 times in a year ) : 178
Number of values outside the 99.99% confidence interval (It should happen once in 4 years) : 122
Number of values outside the 99.999999999999% confidence interval (It should happen twice in a million years): 6
